## Potential Talent

### **Context:**

As a **talent sourcing and management company**, we are interested in **finding talented individuals** for sourcing these candidates to technology companies. **Finding talented candidates is not easy**, for **several reasons**. The **first** reason is one needs to understand what the role is very well to fill in that spot, this requires understanding the client’s needs and what they are looking for in a potential candidate. The **second** reason is one needs to understand what makes a candidate shine for the role we are in search for. **Third**, where to find talented individuals is another challenge.

The nature of our job requires a lot of human labor and is full of **manual operations**. Towards **automating this process** we want to build a better approach that could save us time and finally help us spot potential candidates that could fit the roles we are in search for. Moreover, going beyond that for a specific role we want to fill in we are interested in developing a machine learning powered pipeline that could spot talented individuals, and rank them based on their fitness.

We are right now semi-automatically sourcing a few candidates, therefore the sourcing part is not a concern at this time but we expect to first determine best matching candidates based on how fit these candidates are for a given role. We generally make these searches based on some keywords such as “full-stack software engineer”, “engineering manager” or “aspiring human resources” based on the role we are trying to fill in. These keywords might change, and you can expect that specific keywords will be provided to you.

Assuming that we were able to list and rank fitting candidates, we then employ a review procedure, as each candidate needs to be reviewed and then determined how good a fit they are through manual inspection. This procedure is done manually and at the end of this manual review, we might choose not the first fitting candidate in the list but maybe the 7th candidate in the list. If that happens, we are interested in being able to re-rank the previous list based on this information. This supervisory signal is going to be supplied by starring the 7th candidate in the list. Starring one candidate actually sets this candidate as an ideal candidate for the given role. Then, we expect the list to be re-ranked each time a candidate is starred.

#### Data Description:

The data comes from our sourcing efforts. We removed any field that could directly reveal personal details and gave a unique identifier for each candidate.

#### Attributes:
**id** : unique identifier for candidate (numeric)

**job_title** : job title for candidate (text)

**location** : geographical location for candidate (text)

**connections** : number of connections candidate has, 500+ means over 500 (text)

**Output (desired target)**:
fit - how fit the candidate is for the role? (numeric, probability between 0-1)

Keywords: “Aspiring human resources” or “seeking human resources”

#### Download Data:

https://docs.google.com/spreadsheets/d/117X6i53dKiO7w6kuA1g1TpdTlv1173h_dPlJt5cNNMU/edit?usp=sharing

#### Goal(s):

Predict how fit the candidate is based on their available information (variable fit)

Success Metric(s):

Rank candidates based on a fitness score.

Re-rank candidates when a candidate is starred.

#### Bonus(es):

We are interested in a robust algorithm, tell us how your solution works and show us how your ranking gets better with each starring action.

How can we filter out candidates which in the first place should not be in this list?

Can we determine a cut-off point that would work for other roles without losing high potential candidates?

Do you have any ideas that we should explore so that we can even automate this procedure to prevent human bias?

In [18]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import linear_kernel
pd.options.display.max_columns = 30

In [19]:
df = pd.read_csv('potential-talents - Aspiring human resources - seeking human resources.csv').set_index('id')
df.head(20)

,job_title,location,connection,fit
id,,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
7,Student at Humber College and Aspiring Human R...,Kanada,61,NaN
8,HR Senior Specialist,San Francisco Bay Area,500+,NaN
9,Student at Humber College and Aspiring Human R...,Kanada,61,NaN


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 1 to 104
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   job_title   104 non-null    object 
 1   location    104 non-null    object 
 2   connection  104 non-null    object 
 3   fit         0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 4.1+ KB


In [21]:
df[df.job_title.str.contains("Human Resources")]

,job_title,location,connection,fit
id,,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
7,Student at Humber College and Aspiring Human R...,Kanada,61,NaN
9,Student at Humber College and Aspiring Human R...,Kanada,61,NaN
10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN
13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",500+,NaN
14,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
15,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN


In [43]:
for i in range(len(df)):
    print(i+1, " ", df.job_title.iloc[i])

1   2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional
2   Native English Teacher at EPIK (English Program in Korea)
3   Aspiring Human Resources Professional
4   People Development Coordinator at Ryan
5   Advisory Board Member at Celal Bayar University
6   Aspiring Human Resources Specialist
7   Student at Humber College and Aspiring Human Resources Generalist
8   HR Senior Specialist
9   Student at Humber College and Aspiring Human Resources Generalist
10   Seeking Human Resources HRIS and Generalist Positions
11   Student at Chapman University
12   SVP, CHRO, Marketing & Communications, CSR Officer | ENGIE | Houston | The Woodlands | Energy | GPHR | SPHR
13   Human Resources Coordinator at InterContinental Buckhead Atlanta
14   2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional
15   2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources pro

In [44]:
df.job_title.value_counts()

2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional                 7
Student at Humber College and Aspiring Human Resources Generalist                                                        7
Aspiring Human Resources Professional                                                                                    7
People Development Coordinator at Ryan                                                                                   6
Aspiring Human Resources Specialist                                                                                      5
Native English Teacher at EPIK (English Program in Korea)                                                                5
HR Senior Specialist                                                                                                     5
Advisory Board Member at Celal Bayar University                                                                          4
Human Resources 

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer object
vectorizer = TfidfVectorizer()



In [48]:
# Vectorizing pre-processed movie plots using TF-IDF
tfidfvec = TfidfVectorizer()
tfidf_job = tfidfvec.fit_transform((df["job_title"]))
  
# Finding cosine similarity between vectors
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_job, tfidf_job)

In [ ]:
def best_worst(n, by, ascending = False, min_bud = 0, min_votes = 0):
    
    
    df2 = df_best.loc[(df_best.Budget >= min_bud) & (df_best.Votes >= min_votes), 
                      ["", by]].sort_values(by = by, ascending = ascending).head(n).copy()
    
    return HTML(df2.to_html(escape=False))